In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_json('data/Movies_and_TV_5.json', lines=True)
X = df['reviewText'].values
y = df['overall'].values
print(f"X.shape: {X.shape}")
print(f"y.shape: {y.shape}")
y = y - 1
# set(y)

In [ ]:
X = df['reviewText'].to_numpy(dtype='<U2000')

In [ ]:
VOCAB_SIZE = 30000
tf.keras.backend.clear_session()
binary_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='count',
    pad_to_max_tokens=True
)
binary_vectorizer.adapt(X)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    binary_vectorizer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [ ]:
model.fit(X,y, epochs=50)

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))